In [16]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

database = pd.read_hdf(r"C:\dev\2408_SU24_F31\processed\database.h5")

In [ ]:
# gather ROI metadata (iterable over planes)

stat = database.raw.stat
ops = database.raw.ops

# index stat and ops by subject/session
if isinstance(ops, dict):
    stat_list = [stat]
    ops_list = [ops]
else:
    stat_list = stat
    ops_list = ops

for p in range(len(ops_list)):
    stat_p = stat_list[p]
    ops_p = ops_list[p]

    ncells = len(stat_p)
    im = np.zeros((ops_p['Ly'], ops_p['Lx']))
    for n in range(ncells):
        ypix = stat_p[n]['ypix'][~stat_p[n]['overlap']]
        xpix = stat_p[n]['xpix'][~stat_p[n]['overlap']]
        im[ypix, xpix] = n + 1

    plt.imshow(im)
    plt.title(f"session {p} – {ncells} ROIs")
    plt.show()

In [ ]:
# heatmap of ROI activity
import seaborn as sns

sns.heatmap(database.calculate.deltaf_f.loc['sub-SB03','ses-01'], cmap='Reds', cbar_kws={'label': 'ΔF/F'})
plt.show()

In [ ]:
def plot_poster_trace(database, subject, session, roi_idx, num_points=6000):
    """
    Create a poster-quality ΔF/F trace for one ROI.
    
    Args:
      database   : main DataFrame
      subject    : e.g. 'sub-SB03'
      session    : e.g. 'ses-01'
      roi_idx    : ROI index (e.g. 61)
      num_points : number of frames to plot (default: 1000)
    
    Returns:
      fig, ax : Matplotlib figure and axes
    """
    import matplotlib.pyplot as plt
    # extract data
    times = database.toolkit.timestamps.loc[subject, session].values[:num_points]
    trace = database.calculate.smoothed_dff.loc[subject, session][roi_idx][:num_points]

    # poster styling
    plt.rcParams.update({
        'figure.figsize': (8, 4),
        'figure.dpi': 600,
        'font.size': 16,
        'font.family': 'Arial',
        'axes.linewidth': 2,
        'axes.labelweight': 'bold',
        'axes.titlesize': 18,
        'xtick.major.size': 8,
        'xtick.major.width': 2,
        'ytick.major.size': 8,
        'ytick.major.width': 2,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14,
    })

    fig, ax = plt.subplots()
    ax.plot(times, trace, color='#2E86AB', linewidth=3)

    # labels & title
    ax.set_xlabel('Time (s)', labelpad=10, weight='bold')
    ax.set_ylabel('ΔF/F', labelpad=10, weight='bold')
    ax.set_title(f'{subject} {session} – ROI {roi_idx}', pad=15, weight='bold')

    # thicken spines
    for spine in ax.spines.values():
        spine.set_linewidth(2)

    plt.tight_layout()
    return fig, ax

plot_poster_trace(database, 'sub-SB03', 'ses-01', 38)

In [ ]:
def compute_roi_tuning(data, subject, session, roi_idx,
                       blank_duration=3.0, stim_duration=2.0):
    """
    Compute a tuning curve for one ROI by measuring its mean ΔF/F
    in the grating window (blank_duration→blank_duration+stim_duration)
    for each orientation across all blocks.

    Returns:
      orientations : list of unique orientations in cycle order
      mean_resps   : array of shape (n_orientations,)
      sem_resps    : array of shape (n_orientations,)
      block_pref   : list of preferred orientation per block
    """
    # pull trial table for this subject/session
    trials = data[('toolkit','trials')].loc[(subject, session)]
    if trials.empty:
        raise ValueError(f"No trials for {subject} {session}")

    # time & dff arrays per trial
    orientations = trials['orientation'].values
    times = trials['time'].values
    dffs = trials['dff'].values  # array of shape (n_trials, n_rois, n_time)

    # window indices for the grating (2s) period
    t0 = blank_duration
    t1 = blank_duration + stim_duration
    
    # use first trial's time vector for indexing
    tvec = np.array(times[0])
    mask = (tvec >= t0) & (tvec < t1)

    # collect responses per trial: mean ΔF/F over stim window
    resp = np.array([dff[roi_idx, mask].mean() for dff in dffs])

    # unique orientations in presented order
    uniq_oris = np.unique(orientations)
    mean_resps = []
    sem_resps = []
    for ori in uniq_oris:
        sel = resp[orientations == ori]
        mean_resps.append(sel.mean())
        sem_resps.append(sel.std(ddof=1) / np.sqrt(len(sel)))

    # preferred orientation per block
    n_blocks = len(resp) // len(uniq_oris)
    block_pref = []
    for b in range(n_blocks):
        block_resp = resp[b*len(uniq_oris):(b+1)*len(uniq_oris)]
        block_pref.append(uniq_oris[np.argmax(block_resp)])

    return np.array(uniq_oris), np.array(mean_resps), np.array(sem_resps), block_pref

test = compute_roi_tuning(database, 'sub-SB03', 'ses-01', roi_idx=38)

print(test)

In [ ]:
# Calculate tuning vector for each ROI in a given subject,session pair
from scipy.signal import find_peaks

print(database.toolkit.trials.loc['sub-SB03','ses-01']['direction'], database.toolkit.trials.loc['sub-SB03','ses-01']['orientation'])
print(database.toolkit.trials.loc['sub-SB03','ses-01']['dff_on'])

def compute_tuning_vectors(database, subject, session):
    """
    Compute tuning vectors for all ROIs in one subject/session pair.

    Returns:
      tuning_vectors : array of shape (n_rois, n_orientations)
      dir_vectors    : array of shape (n_rois, n_directions)
    """
    # pull trial table for this subject/session
    trials = database[('toolkit','trials')].loc[(subject, session)]

    # define variables
    dir = trials['direction'].values
    ori = trials['orientation'].values
    times = trials['time'].values 
    dff_on = trials['dff'].values  # array of shape (n_trials, n_rois, n_time)
    num_peaks = find_peaks(dff_on, prominence=0.3)

    ori_tuning = {'dir': dir, 'ori': ori, 'times': times, 'dff_on': dff_on, 'num_peaks': num_peaks}

    orientations = np.unique(ori)
    directions = np.unique(dir)

    orientation_means = np.zeros(len(dff_on))
    for ori in orientations:
        orientation_means.append(n)


    return ori_tuning

tuning = compute_tuning_vectors(database, 'sub-SB03', 'ses-01')
print(tuning)




    



0        0
1       45
2       90
3      135
4      180
      ... 
115    135
116    180
117    225
118    270
119    315
Name: direction, Length: 120, dtype: int64 0        0
1       45
2       90
3      135
4        0
      ... 
115    135
116      0
117     45
118     90
119    135
Name: orientation, Length: 120, dtype: int64
0      [[0.9721103038662243, 0.9219389144805489, 0.94...
1      [[0.7257215296337799, 0.9109142883804063, 0.91...
2      [[0.6886325388276373, 0.7597412579009271, 0.97...
3      [[1.0498814043526887, 1.345078998219768, 1.381...
4      [[1.1425870933490123, 0.9667170132266651, 0.88...
                             ...                        
115    [[0.3537536144848294, 0.2121080618246956, 0.30...
116    [[0.5661237386188644, 0.4890623022280237, 0.35...
117    [[0.3716128229097406, 0.3540401037350839, 0.50...
118    [[0.33513585056494527, 0.4193337067986303, 0.4...
119    [[0.4089657045987031, 0.3935834318970558, 0.40...
Name: dff_on, Length: 120, dtype: object


ValueError: setting an array element with a sequence.

In [ ]:
# OSI, DSI calculation

def calculate_osi_dsi(responses, angles):
    """
    Calculate Orientation Selectivity Index (OSI) and Direction Selectivity Index (DSI).
    
    Args:
      responses : array-like of responses to different angles
      angles    : array-like of angles in degrees (same length as responses) 
      """
    angles_rad = np.deg2rad(angles)
    vector_sum = np.sum(responses * np.exp(1j * angles_rad))
    vector_sum_180 = np.sum(responses * np.exp(1j * 2 * angles_rad))
    
    dsi = np.abs(vector_sum) / np.sum(responses) if np.sum(responses) != 0 else 0
    osi = np.abs(vector_sum_180) / np.sum(responses) if np.sum(responses) != 0 else 0
    
    return osi, dsi


# tc_norm is normalized vector of responses
# angle_rad is orientation angle in radians

tc_ori = np.array([0.1, 0.8, 1.0, 0.5])
angle_ori = np.deg2rad([0, 45, 90, 135])

tc_dir = np.array([0.1, 0.8, 1.0, 0.5, , 0.8, 1.0, 0.5])
angle_dir = np.deg2rad([0, 45, 90, 135, 180, 225, 270, 315])

osi = np.abs(np.sum(tc_ori*np.exp(2j* angle_ori)) / np.sum(tc_ori))

dsi = np.abs(np.sum(tc_dir*np.exp(1j* angle_dir)) / np.sum(tc_dir))

print(osi)
print(dsi)

0.39528470752104744
0.1578947368421053


In [ ]:
# Plot tuning curve
def plot_tuning_curve(angles, responses):
    """
    Plot tuning curve with polar plot.
    
    Args:
      angles    : array-like of angles in degrees
      responses : array-like of responses (same length as angles)
    """
    import matplotlib.pyplot as plt
    import numpy as np

    angles_rad = np.deg2rad(angles)
    
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.plot(angles_rad, responses, marker='o', linestyle='-')
    
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    
    ax.set_xticks(np.deg2rad(np.arange(0, 360, 45)))
    ax.set_xticklabels([f'{i}°' for i in range(0, 360, 45)])
    
    ax.set_title('Tuning Curve', va='bottom')
    
    plt.tight_layout()
    plt.show()